## Simulations

In this notebook we wish to run a simulation demonstrating some of the basic claims we make regarding the random 
forest. 

The key claims we would like to demonstrate are thus:

- A dataset can have heirarchal behavior
    - an RF will identify such hierarchal structure 
    - an RF will capture local changes in covariance etc
    
    - A PCA CANNOT capture some of the effects that we will identify as local in distinct PCs.

- When a dataset undergoes changes in population prevalence, we identify this as a shift in factor values

- When a dataset undergoes a change in population behavior we identify this as a shift in predictive power

To reflect a hierarchal structure with meaningful local behavior, we will need several features that have different means among different clusters, but importantly also interact with each other. In order to approximately reflect the behavior of single-cell data we will draw from a mixture of multi-dimensional gaussians with known covariance and then randomly sample over the draw as per poisson. 

Let's operate on 10 features total. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from scipy.stats import multivariate_normal,norm
from sklearn.datasets import make_blobs
from sklearn.manifold import TSNE


In [ ]:
# First we generate the macro-structure. 

# We will produce a simple pattern with 5 features with a mean and covariance, 3 features with only a mean, 
# and two features of pure noise

macro_cov = np.array(
    [
        [1, 0, 1, 0, 2, 0, 0, 0, 0, 0],
        [0, 1,-1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 2, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 2, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 3, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    ]
)

# We want a positive semi-definite (or definite) matrix
macro_cov = np.dot(macro_cov,macro_cov.T)



macro_mean_1 = [
                    2,3,1,1,0, 
                    0,1,2, 
                    3,3
]

macro_mean_2 = [
                    0,-1,1,0,0, 
                    -3,2,2, 
                    3,3
]





In [ ]:
macro_cov

In [ ]:
macro_samples_1 = multivariate_normal(macro_mean_1,macro_cov).rvs(1000)
macro_samples_2 = multivariate_normal(macro_mean_2,macro_cov).rvs(2000)

coordinates = np.vstack([macro_samples_1,macro_samples_2])

In [ ]:

t_coordinates = TSNE().fit_transform(coordinates)

plt.figure()
plt.scatter(*t_coordinates.T)

In [ ]:
# To introduce a local effect, we would like to use a similar but not identical covariance matrix for a smaller part of the dataset

micro_cov_1 = np.array(
    [
        [1, 0, 1, 0, 2, 0, 0, 2, 0, 0],
        [0, 1,-1, 0, 0, 0, 0, 0, 3, 0],
        [0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 2, 0, 0, 0, 2, 0, 0],
        [0, 0, 0, 0, 1, 0, 1, 2, 0, 0],
        [0, 0, 0, 0, 0, 2, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 2, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 3, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    ]
)
micro_cov_1 = np.dot(micro_cov_1,micro_cov_1.T)

micro_cov_2 = np.array(
    [
        [1, 0, 1, 0, 2, 0, 0,-1, 0, 0],
        [0, 1,-1, 0, 0, 0, 0, 0, 3, 0],
        [0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 2, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 1, 2, 0, 0],
        [0, 0, 0, 0, 0, 2, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1,-1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 2, 2, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 3, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    ]
)

micro_cov_2 = np.dot(micro_cov_2,micro_cov_2.T)

micro_samples_1 = multivariate_normal(macro_mean_2,micro_cov_1).rvs(2000)
micro_samples_2 = multivariate_normal(macro_mean_1,micro_cov_2).rvs(1000)


In [ ]:
coordinates = np.vstack([micro_samples_1,micro_samples_2])

t_coordinates = TSNE().fit_transform(coordinates)

plt.figure()
plt.scatter(*t_coordinates.T)
plt.show()

In [ ]:
colors = np.zeros(3000)
colors[:1000] = 1
colors[1000:] = 2
# colors[-500:] = 3

In [ ]:

plt.figure()
plt.scatter(*t_coordinates.T,c=colors)
plt.show()

In [ ]:
import sys
sys.path.append('../src/')
import tree_reader as tr 
import lumberjack

In [ ]:
forest = lumberjack.fit(
    coordinates,
    trees=100,
    ifs=5,
    ofs=5,
    braids=1,
    ss=200,
    leaves=10,
    depth=3,
    reduce_input='false',
    reduce_output='false'
)

In [ ]:
forest.tsne_coordinates = t_coordinates

In [ ]:
forest.interpret_splits(mode='additive_mean',metric='cosine',pca=0,k=100)

In [ ]:
forest.maximum_spanning_tree(mode='samples')

In [ ]:
forest.html_tree_summary(n=5)

In [ ]:
# First we will generate the macro-structure. We will generate an eigenvector that applies globally, has a 
# multivariate normal set of loadings and a bimodal normal distribution of values

global_noise = [
    1,1,1,1,1,
    1,1,1,
    1,1,
]

loading_means_global = [
        1,0,-2,3,5,
        0,0,0,
        3,3
    ]
    
    
noise = multivariate_normal(global_noise,np.identity(10)).rvs(3000)    
loadings = multivariate_normal(loading_means_global,np.identity(10)).rvs(3000)
score_draws = norm().rvs(3000) * 3

score_draws[:1000] += 3

coordinates = np.dot(score_draws,loadings) + noise


In [ ]:
# local_loading_means_1 = [
#     0,2,0,0,0,
#     1,1,1,
#     0,0,
# ]

# local_loading_means_2 = [
#     0,-1,0,0,0,
#     1,1,2,
#     0,0,
# ]

# local_loadings_1 = multivariate_normal(local_loading_means_1,np.identity(10)/3).rvs(1500)
# local_loadings_2 = multivariate_normal(local_loading_means_2,np.identity(10)/3).rvs(500)

# local_scores_1 = np.dot(norm().rvs(1500),local_loadings_1)
# local_scores_2 = np.dot(norm().rvs(500),local_loadings_2)

# coordinates[1000:2500] += local_scores_1
# coordinates[2500:] += local_scores_2

local_marker = np.zeros(3000,dtype=bool)
local_marker[1000:] = True

In [ ]:
t_coordinates = TSNE().fit_transform(coordinates)

plt.figure()
plt.scatter(*t_coordinates.T)
plt.show()

plt.figure()
plt.scatter(*t_coordinates.T,c=local_marker)
plt.show()

In [ ]:
from sklearn.decomposition import PCA

model = PCA().fit(coordinates)

In [ ]:
model.explained_variance_ratio_

In [ ]:
model.components_

In [ ]:
pct = model.transform(coordinates)
pct.shape

In [ ]:
plt.figure()
plt.scatter(*t_coordinates.T,c=pct[:,0],cmap='bwr')
plt.colorbar()
plt.show()

plt.figure()
plt.scatter(*t_coordinates.T,c=pct[:,1],cmap='bwr')
plt.colorbar()
plt.show()

plt.figure()
plt.scatter(*t_coordinates.T,c=pct[:,2],cmap='bwr')
plt.colorbar()
plt.show()